# Generate a prettier plot with statistics on the plot

In [ ]:
library(repr)
library(ggpubr)
library(tidyverse)

In [ ]:
config <- list('caudate'= '../../../caudate/_m/genes/diffExpr_maleVfemale_full.txt',
               'dlpfc'= '../../../dlpfc/_m/genes/diffExpr_maleVfemale_full.txt', 
               'hippo'= '../../../hippocampus/_m/genes/diffExpr_maleVfemale_full.txt')

In [ ]:
save_ggplots <- function(fn, p, w, h){
    for(ext in c('.pdf', '.png', '.svg')){
        ggsave(paste0(fn, ext), plot=p, width=w, height=h)
    }
}

get_deg <- function(fn){
    dft <- read.delim(fn, row.names=1)
    dft['Feature'] = row.names(dft)
    dft['Dir'] = sign(dft['t'])
    if('gene_id' %in% colnames(dft)){
        dft['ensemblID'] <- gsub('\\..*', '', dft$gene_id)
    }
    return(subset(dft, select=c('Feature', 'ensemblID', 'adj.P.Val', 'logFC', 't', 'Dir')))
}

get_deg_sig <- function(fn, fdr){
    dft <- get_deg(fn)
    return(subset(dft, adj.P.Val < fdr))
}

merge_dataframe <- function(tissue1, tissue2){
    return(merge(get_deg(config[[tissue1]]), get_deg(config[[tissue2]]), 
                 by='Feature', suffixes=c(paste0('_',tissue1), paste0('_',tissue2))))
}

merge_dataframes_sig <- function(tissue1, tissue2){
    fdr = 0.05
    #fdr2 = ifelse(tissue2 != 'dlpfc', 0.05, 0.01)
    return(merge(get_deg_sig(config[[tissue1]], fdr), get_deg_sig(config[[tissue2]], fdr), 
                 by='Feature', suffixes=c(paste0('_',tissue1), paste0('_',tissue2))))
}

tissue_annotation <- function(tissue){
    return(list('dlpfc'='DLPFC', 'hippo'='Hippocampus', 'caudate'='Caudate')[[tissue]])
}

get_scatter_plot <- function(tissue1, tissue2, merge_fnc, coords){
    dft <- merge_fnc(tissue1, tissue2)
    sp = ggscatter(dft, x=paste0('t_', tissue1), y=paste0('t_', tissue2), add="reg.line", 
                   xlab=paste0('T-statistic (',tissue_annotation(tissue1), ')'),
                   ylab=paste0('T-statistic (',tissue_annotation(tissue2), ')'),
                   add.params = list(color = "blue", fill = "lightgray"), conf.int = FALSE, 
                   cor.method = "pearson", cor.coef = FALSE, cor.coef.size = 7, 
                   cor.coeff.args = list(label.sep = "\n"),  #cor.coef.coord = coords, 
                   ylim=c(-125, 125)) + 
    stat_cor(aes(label = paste0(..rr.label..)), size=8,method = "pearson") + 
    font("xylab", size = 20, face='bold') + font("xy.text", size = 18)
    return(sp)
}

## Genes

In [ ]:
options(repr.plot.width=18, repr.plot.height=6)

In [ ]:
sp1_sig = get_scatter_plot('caudate', 'dlpfc', merge_dataframes_sig, c(-110, 85))
sp2_sig = get_scatter_plot('caudate', 'hippo', merge_dataframes_sig, c(-110, 85))
sp3_sig = get_scatter_plot('dlpfc', 'hippo', merge_dataframes_sig, c(-110, 85))
fig1 = ggarrange(sp1_sig, sp2_sig, sp3_sig, ncol=3, align='v')
fig1 = annotate_figure(fig1, bottom=text_grob("Genes", face = "bold.italic", size = 28))
print(fig1)

In [ ]:
sp1 = get_scatter_plot('caudate', 'dlpfc', merge_dataframe, c(-110, 85))
sp2 = get_scatter_plot('caudate', 'hippo', merge_dataframe, c(-110, 85))
sp3 = get_scatter_plot('dlpfc', 'hippo', merge_dataframe, c(-110, 85))
fig2 = ggarrange(sp1, sp2, sp3, ncol=3, align='v')
fig2 = annotate_figure(fig2, bottom=text_grob("Genes", face = "bold.italic", size = 28))
print(fig2)

In [ ]:
save_ggplots("genes_tstatistic_corr_sig", fig1, 18, 6)
save_ggplots("genes_tstatistic_corr", fig2, 18, 6)

## Transcripts

In [ ]:
config <- list('caudate'= '../../../caudate/_m/transcripts/diffExpr_maleVfemale_full.txt',
               'dlpfc'= '../../../dlpfc/_m/transcripts/diffExpr_maleVfemale_full.txt', 
               'hippo'= '../../../hippocampus/_m/transcripts/diffExpr_maleVfemale_full.txt')

In [ ]:
sp1_sig = get_scatter_plot('caudate', 'dlpfc', merge_dataframes_sig, c(-110, 85))
sp2_sig = get_scatter_plot('caudate', 'hippo', merge_dataframes_sig, c(-110, 85))
sp3_sig = get_scatter_plot('dlpfc', 'hippo', merge_dataframes_sig, c(-110, 85))

In [ ]:
fig3 = ggarrange(sp1_sig, sp2_sig, sp3_sig, ncol=3, align='v')
fig3 = annotate_figure(fig3, bottom=text_grob("Transcripts", face = "bold.italic", size = 28))
print(fig3)

In [ ]:
sp1 = get_scatter_plot('caudate', 'dlpfc', merge_dataframe, c(-110, 85))
sp2 = get_scatter_plot('caudate', 'hippo', merge_dataframe, c(-110, 85))
sp3 = get_scatter_plot('dlpfc', 'hippo', merge_dataframe, c(-110, 85))
fig4 = ggarrange(sp1, sp2, sp3, ncol=3, align='v')
fig4 = annotate_figure(fig4, bottom=text_grob("Transcripts", face = "bold.italic", size = 28))
print(fig4)

In [ ]:
save_ggplots("tx_tstatistic_corr_sig", fig1, 18, 6)
save_ggplots("tx_tstatistic_corr", fig2, 18, 6)

## Junctions

In [ ]:
config <- list('caudate'= '../../../caudate/_m/junctions/diffExpr_maleVfemale_full.txt',
               'dlpfc'= '../../../dlpfc/_m/junctions/diffExpr_maleVfemale_full.txt', 
               'hippo'= '../../../hippocampus/_m/junctions/diffExpr_maleVfemale_full.txt')

In [ ]:
sp1_sig = get_scatter_plot('caudate', 'dlpfc', merge_dataframes_sig, c(-110, 85))
sp2_sig = get_scatter_plot('caudate', 'hippo', merge_dataframes_sig, c(-110, 85))
sp3_sig = get_scatter_plot('dlpfc', 'hippo', merge_dataframes_sig, c(-110, 85))
fig5 = ggarrange(sp1_sig, sp2_sig, sp3_sig, ncol=3, align='v')
fig5 = annotate_figure(fig5, bottom=text_grob("Junctions", face = "bold.italic", size = 28))
print(fig5)

In [ ]:
sp1 = get_scatter_plot('caudate', 'dlpfc', merge_dataframe, c(-110, 85))
sp2 = get_scatter_plot('caudate', 'hippo', merge_dataframe, c(-110, 85))
sp3 = get_scatter_plot('dlpfc', 'hippo', merge_dataframe, c(-110, 85))
fig6 = ggarrange(sp1, sp2, sp3, ncol=3, align='v')
fig6 = annotate_figure(fig6, bottom=text_grob("Junctions", face = "bold.italic", size = 28))
print(fig6)

In [ ]:
save_ggplots("jxn_tstatistic_corr_sig", fig3, 18, 6)
save_ggplots("jxn_tstatistic_corr", fig4, 18, 6)

## Exons

In [ ]:
config <- list('caudate'= '../../../caudate/_m/exons/diffExpr_maleVfemale_full.txt',
               'dlpfc'= '../../../dlpfc/_m/exons/diffExpr_maleVfemale_full.txt', 
               'hippo'= '../../../hippocampus/_m/exons/diffExpr_maleVfemale_full.txt')

In [ ]:
sp1_sig = get_scatter_plot('caudate', 'dlpfc', merge_dataframes_sig, c(-110, 85))
sp2_sig = get_scatter_plot('caudate', 'hippo', merge_dataframes_sig, c(-110, 85))
sp3_sig = get_scatter_plot('dlpfc', 'hippo', merge_dataframes_sig, c(-110, 85))
fig7 = ggarrange(sp1_sig, sp2_sig, sp3_sig, ncol=3, align='v')
fig7 = annotate_figure(fig7, bottom=text_grob("Exons", face = "bold.italic", size = 28))
print(fig7)

In [ ]:
sp1 = get_scatter_plot('caudate', 'dlpfc', merge_dataframe, c(-110, 85))
sp2 = get_scatter_plot('caudate', 'hippo', merge_dataframe, c(-110, 85))
sp3 = get_scatter_plot('dlpfc', 'hippo', merge_dataframe, c(-110, 85))
fig8 = ggarrange(sp1, sp2, sp3, ncol=3, align='v')
fig8 = annotate_figure(fig8, bottom=text_grob("Exons", face = "bold.italic", size = 28))
print(fig8)

In [ ]:
save_ggplots("exon_tstatistic_corr_sig", fig5, 18, 6)
save_ggplots("exon_tstatistic_corr", fig6, 18, 6)

## Arrange plots

In [ ]:
options(repr.plot.width=18, repr.plot.height=24)
figure_sig = ggarrange(fig1,fig3,fig7,fig5,ncol=1,align='h', labels='AUTO', 
                       font.label=list(size=36,face='bold'))
figure_sig

In [ ]:
figure_all = ggarrange(fig2,fig4,fig8,fig6,ncol=1,align='h', labels='AUTO', 
                       font.label=list(size=36,face='bold'))
figure_all

In [ ]:
save_ggplots("feature_tstatistic_corr_sig", figure_sig, 18, 24)
save_ggplots("feature_tstatistic_corr", figure_all, 18, 24)

## Reproducibility Information

In [ ]:
print("Reproducibility Information:")
Sys.time()
proc.time()
options(width=120)
sessioninfo::session_info()